In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import feather
import datetime
from matplotlib_venn import venn2
from sklearn import preprocessing
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
import lightgbm as lgbm
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
pd.set_option('display.max_Columns', 100)

In [2]:
cartlog = pd.read_feather('../inputs/cartlog.f')
product_master = pd.read_feather('../inputs/product_master.f')
meta = pd.read_feather('../inputs/meta.f')
user_master = pd.read_feather('../inputs/user_master.f')
test = pd.read_csv('../inputs/test.csv')
display_action_id = pd.read_csv('../inputs/display_action_id.csv')

product_master['JAN'] = product_master['JAN'].astype(str)

In [3]:
# le = preprocessing.LabelEncoder()
# display_action_id['display_name'] = le.fit_transform(display_action_id['display_name'])
# display_action_id['action_name'] = le.fit_transform(display_action_id['action_name'])
# display_action_id.drop(columns=['論理名'], inplace=True)
# cartlog = pd.merge(cartlog, display_action_id, on='display_action_id', how='left')

In [4]:
test_sessions = test["session_id"].unique()
print(len(test_sessions))
test_input_log = cartlog[cartlog["session_id"].isin(test_sessions)]

56486


In [5]:
class RetailDataset:
    def __init__(self, thres_sec, cartlog, product, meta):
        self.thres_sec = thres_sec
        self.cartlog = cartlog
        self.product_master = product
        self.meta = meta.copy()
        
        meta['time_elapsed_sec'] = meta['time_elapsed'] * 60
        self.meta.loc[self.meta['time_elapsed_sec'].isnull(), 'time_elapsed_sec'] = thres_sec
        self.target_category_ids = [
            38,  # アイスクリーム__ノベルティー
            110,  # スナック・キャンディー__ガム
            113,  # スナック・キャンディー__シリアル
            114,  # スナック・キャンディー__スナック
            134,  # チョコ・ビスクラ__チョコレート
            171,  # ビール系__RTD
            172,  # ビール系__ノンアルコール
            173,  # ビール系__ビール系
            376,  # 和菓子__米菓
            435,  # 大型PET__無糖茶（大型PET）
            467,  # 小型PET__コーヒー（小型PET）
            537,  # 水・炭酸水__大型PET（炭酸水）
            539,  # 水・炭酸水__小型PET（炭酸水）
            629,  # 缶飲料__コーヒー（缶）
            768,  # 麺類__カップ麺
        ]

    
    def get_log_first_half(self) -> pd.DataFrame:
        """以下の条件を満たすログを取得する
        - 学習期間(2020-08-01の前日まで)のセッションである
        """
        first_half_sessions = set(
            self.meta.query("date < '2020-08-01'")["session_id"].unique()
        )
        return self.cartlog[self.cartlog["session_id"].isin(first_half_sessions)]

    def get_train_output_log(self) -> pd.DataFrame:
        """以下の条件を満たすログを取得する
        - 学習期間(2020-08-01の前日まで)のセッションである
        - 指定した時間(thres_sec)以降にログが存在している
        """
        return pd.merge(
            self.get_log_first_half(),
            self.meta[["session_id", "time_elapsed_sec"]],
            on=["session_id"],
            how="inner",
        ).query("spend_time > time_elapsed_sec")

    def get_train_sessions(self) -> set:
        """以下の条件を満たすセッションを取得する
        - 学習期間(2020-08-01の前日まで)のセッションである
        - 指定した時間(thres_sec)以降にログが存在している
        """
        return set(self.get_train_output_log()["session_id"].unique())

    def get_train_input_log(self) -> pd.DataFrame:
        """以下の条件を満たすログを取得する
        - 学習期間(2020-08-01の前日まで)のセッションである
        - 指定した時間(thres_sec)以降にログが存在している
        - 指定した時間(thres_sec)より前のログである
        """
        train_sessions = self.get_train_sessions()
        return pd.merge(
            self.get_log_first_half()[
                self.get_log_first_half()["session_id"].isin(train_sessions)
            ],
            self.meta[["session_id", "time_elapsed_sec"]],
            on=["session_id"],
            how="inner",
        ).query("spend_time <= time_elapsed_sec")

    def get_payment_sessions(self) -> set:
        """以下の条件を満たすセッションを取得する
        - 決済を行った
        """
        return set(self.cartlog.query("is_payment == 1")["session_id"].unique())

    def agg_payment(self, cartlog) -> pd.DataFrame:
        """セッションごと・商品ごとの購買個数を集計する"""
        # 購買情報は商品のものだけ.
        target_index = cartlog["kind_1"] == "商品"

        # JANコード (vale_1)ごとに商品の購入個数(n_items)を足し算
        agg = (
            cartlog.loc[target_index]
            .groupby(["session_id", "value_1"])["n_items"]
            .sum()
            .reset_index()
        )
        agg = agg.rename(columns={"value_1": "JAN"})
#         agg = agg.astype({"JAN": int})
        return agg

    def get_train_target(self) -> pd.DataFrame:
        """学習で使用するセッションの目的変数を取得する"""
        # 空のターゲット用データフレームを用意する
        train_sessions = self.get_train_sessions()
        train_target = pd.DataFrame(
            np.zeros((len(train_sessions), len(self.target_category_ids))),
            index=train_sessions,
            columns=self.target_category_ids,
        ).astype(int)
        train_target.index.name = "session_id"

        # 集計する
        train_output_log = self.get_train_output_log()
        train_items_per_session_jan = self.agg_payment(train_output_log)
        train_items_per_session_target_jan = pd.merge(
            train_items_per_session_jan,
            self.product_master[["JAN", "category_id"]],
            on="JAN",
            how="inner",
        ).query("category_id in @self.target_category_ids")
        train_target_pos = (
            train_items_per_session_target_jan.groupby(["session_id", "category_id"])[
                "n_items"
            ]
            .sum()
            .unstack()
            .fillna(0)
            .astype(int)
        )
        train_target_pos[train_target_pos > 0] = 1
        train_target_pos[train_target_pos <= 0] = 0

        train_target.loc[train_target_pos.index] = train_target_pos.values
        return train_target[self.target_category_ids].reset_index()

In [6]:
def get_train_log(elapsed_min):
    dataset = RetailDataset(elapsed_min*60, cartlog, product_master, meta)
    train_input_log = dataset.get_train_input_log()
    y_train = dataset.get_train_target()
    print('train_session', y_train.shape)
    return train_input_log, y_train

In [7]:
ELAPSED_MIN = [0, 3, 5, 10]

In [8]:
def save_train_log():
    for elap_min in ELAPSED_MIN:
        train_log, train_y = get_train_log(elap_min)
        train_log = train_log.reset_index(drop=True)
        train_y = train_y.reset_index(drop=True)
        train_y.columns = [str(c) for c in train_y.columns]
        train_log.to_feather('../inputs/train_log_{}.f'.format(elap_min))
        train_y.to_feather('../inputs/train_y_{}.f'.format(elap_min))

In [9]:
def load_train_log():
    train_log = {}
    train_y = {}
    for elap_min in ELAPSED_MIN:
        log = pd.read_feather('../inputs/train_log_{}.f'.format(elap_min))
        y = pd.read_feather('../inputs/train_y_{}.f'.format(elap_min))
        train_log[elap_min] = log
        train_y[elap_min] = y
    return train_log, train_y

In [10]:
# save_train_log()

### ユーザ情報

In [11]:
user_features = pd.merge(meta[["session_id", "user_id"]], user_master, on="user_id", how="left").drop(columns=["user_id"])
user_features.loc[user_features['age'] >= 80, 'age'] = np.NaN
user_features.loc[user_features['age'] < 10, 'age'] = np.NaN
user_features.loc[user_features['gender'] > 1, 'gender'] = np.NaN

### メタ情報

In [12]:
meta_features = meta.copy()
meta_features['year'] = meta_features['date'].dt.year
meta_features['month'] = meta_features['date'].dt.month
meta_features['day'] = meta_features['date'].dt.day
meta_features['dow'] = meta_features['date'].dt.dayofweek
meta_features['doy'] = meta_features['date'].dt.dayofyear
le = preprocessing.LabelEncoder()
meta_features['userid'] = le.fit_transform(meta_features['user_id'])
meta_features.drop(columns=['user_id', 'date', 'time_elapsed', 'date_str'], inplace=True)

In [13]:
meta_features.head()

,session_id,hour,register_number,year,month,day,dow,doy,userid
0,0,4,2003,2019,2,14,3,45,29324
1,1,4,2006,2019,2,14,3,45,23972
2,2,5,2001,2019,2,14,3,45,39496
3,3,5,2001,2019,2,14,3,45,12998
4,4,5,2002,2019,2,14,3,45,37186


### セッション単位の特徴量

In [14]:
def get_session_info(input_log):
    n_actions = input_log.groupby(["session_id"]).size().rename("n_actions")
    n_add_items = input_log[input_log['kind_1'] == '商品'].groupby(["session_id"]).size().rename("n_add_items")
    mean_spend_time = input_log.groupby(["session_id"])["spend_time"].mean()
    n_coupon_counts = input_log[input_log['kind_1'] == 'クーポン'].groupby(["session_id"]).size().rename("n_coupon_counts")
    n_recipe_counts = input_log[input_log['kind_1'] == 'レシピ'].groupby(["session_id"]).size().rename("n_recipe_counts")
    n_bland_counts = input_log[input_log['kind_1'] == 'ブランドスイッチ'].groupby(["session_id"]).size().rename("n_bland_counts")

    session_features = pd.concat([
        n_actions,
        n_add_items,
        mean_spend_time,
        n_coupon_counts,
    ], axis=1)
    return session_features.reset_index()

### セッション単位で集計

In [15]:
def get_session_features(input_log):
    df_ses = pd.DataFrame(columns=['session_id'])
    ses_info = get_session_info(input_log)
    df_ses = pd.merge(df_ses, ses_info, on='session_id', how='outer')
    return df_ses

### 特徴量を集約する

In [16]:
def merge_features(train_input_log, session):
    ses_features = get_session_features(train_input_log)
    
    out = pd.DataFrame({"session_id": session})
    out = pd.merge(out, ses_features, on="session_id", how="left")
    out = pd.merge(out, user_features, on="session_id", how="left")
    out = pd.merge(out, meta_features, on="session_id", how="left")
    assert len(session) == len(out)
    return out

In [17]:
def get_train_all_features(elapsed_min, train_log_list, train_y_list):
    train_input_log = train_log_list[elapsed_min]
    y_train = train_y_list[elapsed_min]
    
    train_features = merge_features(train_input_log, y_train['session_id'])
    print('train_session', y_train.shape)
    return train_features, y_train

In [18]:
def get_test_all_feature(elapsed_min):
    test_meta = meta[meta['session_id'].isin(test_sessions)]
    test_meta = test_meta[test_meta['time_elapsed'] == elapsed_min]
    test_input_elapsed = pd.merge(test_input_log, test_meta[['session_id']], on='session_id', how='left')
    
    test_features = merge_features(test_input_elapsed, test_meta['session_id'])
    print('test_session', len(test_meta))
    return test_features

In [19]:
train_log_list, train_y_list = load_train_log()

In [20]:
train_features, y_train = get_train_all_features(10, train_log_list, train_y_list)
train_features

train_session (404500, 16)


,session_id,n_actions,n_add_items,spend_time,n_coupon_counts,age,gender,hour,register_number,year,month,day,dow,doy,userid
0,105,12.0,11.0,270.250000,NaN,40.0,0.0,9,1005,2019,2,14,3,45,9099
1,106,17.0,15.0,324.117647,NaN,60.0,1.0,9,1010,2019,2,14,3,45,25432
2,107,22.0,9.0,247.000000,3.0,30.0,0.0,9,1010,2019,2,14,3,45,37346
3,108,11.0,6.0,194.090909,2.0,60.0,0.0,9,1011,2019,2,14,3,45,32599
4,109,10.0,8.0,290.700000,1.0,70.0,0.0,9,1013,2019,2,14,3,45,26095
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
404495,663697,19.0,15.0,287.157895,2.0,30.0,1.0,23,1028,2020,7,31,4,213,17637
404496,663698,6.0,5.0,269.166667,NaN,50.0,0.0,23,1058,2020,7,31,4,213,24044
404497,663699,3.0,2.0,283.333333,NaN,20.0,0.0,23,1076,2020,7,31,4,213,38635
404498,663702,5.0,4.0,26.000000,NaN,NaN,NaN,23,1101,2020,7,31,4,213,7257


In [21]:
n_fold = 4

In [22]:
lgbm_param = {
    'objective' : 'binary',
    'boosting_type': 'gbdt',
    'metric': 'auc',
    'seed' : 0,
    'learning_rate':  0.1,
    'verbose': -1
}

In [23]:
def train_lgbm(X, y, params=lgbm_param):

    fold = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=0)
    cv = fold.split(X, y)
    
    models = []
    oof_pred = np.zeros_like(y, dtype=np.float)
    
    cat_feat = ['age', 'gender', 'dow']

    for i, (idx_train, idx_valid) in enumerate(cv): 
        x_train, y_train = X.iloc[idx_train], y.iloc[idx_train]
        x_valid, y_valid = X.iloc[idx_valid], y.iloc[idx_valid]
#         print('x_train', x_train.shape)
#         print('x_valid', x_valid.shape)
        
        lgbm_train = lgbm.Dataset(x_train, y_train, categorical_feature = cat_feat)
        lgbm_eval = lgbm.Dataset(x_valid, y_valid, reference=lgbm_train, categorical_feature = cat_feat)
        
        lgbm_model = lgbm.train(params, 
                                                    lgbm_train, 
                                                    valid_sets=lgbm_eval,
                                                    categorical_feature = cat_feat,
                                                    num_boost_round=10000,
                                                    early_stopping_rounds=100,
                                                    verbose_eval=-1)
        y_pred = lgbm_model.predict(x_valid, num_iteration=lgbm_model.best_iteration)
        
        oof_pred[idx_valid] = y_pred
        models.append(lgbm_model)
#         print('--- Fold {} Score: {:.4f} ---'.format(i, roc_auc_score(y_valid, y_pred)))

    score = roc_auc_score(y, oof_pred)
    print('--- FINISHED \ whole score: {:.4f} ---'.format(score))
    return oof_pred, models, score

In [24]:
def predict(models, feature):
    pred_list = []
    for i, model in enumerate(models):
        pred = model.predict(feature, num_iteration = model.best_iteration)
        pred_list.append(pred)
    
    score = np.mean(pred_list, axis=0)
    return score

In [25]:
%%time
train_log_list, train_y_list = load_train_log()
df_pred_all = pd.DataFrame()
for elapsed_min in ELAPSED_MIN:
    print(f'===== {elapsed_min} =====')
    train_features, y_train = get_train_all_features(elapsed_min, train_log_list, train_y_list)
    test_features = get_test_all_feature(elapsed_min)

    df_pred = pd.DataFrame(index=test_features['session_id'])
    train_features.drop(columns=['session_id'], inplace=True)
    test_features.drop(columns=['session_id'], inplace=True)
    
    models_list = []
    for target in y_train.columns:
        if target == 'session_id':
            continue
        
        print(f"---- id = {target} -----")
        oof, models, score = train_lgbm(train_features, y_train[target])
        models_list.append(models)

        pred = predict(models, test_features)
        df_pred[target] = pred
    
    df_pred_all = pd.concat([df_pred_all, df_pred])
    print(len(df_pred_all))

===== 0 =====
train_session (641293, 16)
test_session 14277
---- id = 38 -----


/home/umhrysnb/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
/home/umhrysnb/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[851]	valid_0's auc: 0.721981
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[713]	valid_0's auc: 0.719833
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[1521]	valid_0's auc: 0.720813
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[1078]	valid_0's auc: 0.719907
--- FINISHED \ whole score: 0.7205 ---
---- id = 110 -----


/home/umhrysnb/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
/home/umhrysnb/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[197]	valid_0's auc: 0.661947
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[225]	valid_0's auc: 0.649757
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[247]	valid_0's auc: 0.650978
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[229]	valid_0's auc: 0.662698
--- FINISHED \ whole score: 0.6561 ---
---- id = 113 -----


...

Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[655]	valid_0's auc: 0.63845
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[307]	valid_0's auc: 0.641493
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[382]	valid_0's auc: 0.639576
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[102]	valid_0's auc: 0.6339
--- FINISHED \ whole score: 0.6374 ---
---- id = 629 -----


/home/umhrysnb/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
/home/umhrysnb/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[77]	valid_0's auc: 0.648739
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[39]	valid_0's auc: 0.651964
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[150]	valid_0's auc: 0.65331
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[167]	valid_0's auc: 0.641581
--- FINISHED \ whole score: 0.6462 ---
---- id = 768 -----


/home/umhrysnb/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
/home/umhrysnb/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[132]	valid_0's auc: 0.597319
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[137]	valid_0's auc: 0.598114
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[207]	valid_0's auc: 0.597772
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[309]	valid_0's auc: 0.590865
--- FINISHED \ whole score: 0.5958 ---
56486
CPU times: user 16h 13min 31s, sys: 11min 38s, total: 16h 25min 10s
Wall time: 41min 22s


In [26]:
df_pred_all.head()

,38,110,113,114,134,171,172,173,376,435,467,537,539,629,768
session_id,,,,,,,,,,,,,,,
663721,0.077063,0.014927,0.091886,0.291025,0.093227,0.085738,0.040364,0.122231,0.161770,0.041057,0.019934,0.046886,0.125843,0.047974,0.111959
663761,0.129144,0.015047,0.072337,0.383118,0.170033,0.225860,0.097035,0.085973,0.139145,0.084332,0.022890,0.099800,0.149190,0.005290,0.179236
663763,0.095038,0.016903,0.067676,0.295769,0.157746,0.125029,0.062254,0.079867,0.116335,0.122384,0.031871,0.027532,0.039955,0.005151,0.240177
663775,0.091148,0.014398,0.047638,0.257081,0.116856,0.092949,0.104571,0.150892,0.124276,0.072663,0.022190,0.071375,0.075269,0.007263,0.115393
663778,0.127924,0.018316,0.059093,0.263051,0.116724,0.089212,0.021896,0.096104,0.120746,0.088245,0.050373,0.054364,0.060851,0.041559,0.136644


In [27]:
assert len(df_pred_all) == len(test)

In [28]:
submission = pd.merge(test[['session_id']], df_pred_all.reset_index(), on='session_id', how='inner')
assert len(submission) == len(test)

In [29]:
submission.drop(columns='session_id').to_csv('../outputs/submission.csv', index=False)